# Large-Scale Stochastic Variational GP Regression (CUDA)

## Overview

In this notebook, we'll give an overview of how to use SVGP stochastic variational regression ((https://arxiv.org/pdf/1411.2005.pdf)) to rapidly train using minibatches on the `3droad` UCI dataset with hundreds of thousands of training examples. 

In [1]:
import math
import torch
import tqdm
import pyro
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

## Loading Data

For this example notebook, we'll be using the `song` UCI dataset used in the paper. Running the next cell downloads a copy of the dataset that has already been scaled and normalized appropriately. For this notebook, we'll simply be splitting the data using the first 80% of the data as training and the last 20% as testing.

**Note**: Running the next cell will attempt to download a **~136 MB** file to the current directory.

In [2]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor
    
data = torch.Tensor(loadmat('protein.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

## Creating a DataLoader

The next step is to create a torch `DataLoader` that will handle getting us random minibatches of data. This involves using the standard `TensorDataset` and `DataLoader` modules provided by PyTorch.

In this notebook we'll be using a fairly large batch size of 1024 just to make optimization run faster, but you could of course change this as you so choose.

In [3]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

## Defining the SVGP Model

We now define the GP regression module that, intuitvely, will act as the final "layer" of our neural network. In this case, because we are doing variational inference and *not* exact inference, we will be using an `AbstractVariationalGP`. In this example, because we will be learning the inducing point locations, we'll be using a base `VariationalStrategy` with `learn_inducing_locations=True`.

Because the feature extractor we defined above extracts two features, we'll need to define our grid bounds over two dimensions.

In [4]:
from gpytorch.models import SteinVariationalGP


class GPModel(SteinVariationalGP):
    def __init__(self, inducing_points, likelihood, num_data):
        super().__init__(inducing_points, likelihood=likelihood, num_data=num_data)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    
inducing_points = torch.randn(512, train_x.size(-1))
likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()
model = GPModel(inducing_points=inducing_points, likelihood=likelihood, num_data=train_n).cuda()

## Training the Model

The cell below trains the model above, learning both the hyperparameters of the Gaussian process **and** the parameters of the neural network in an end-to-end fashion using Type-II MLE.

Unlike when using the exact GP marginal log likelihood, performing variational inference allows us to make use of stochastic optimization techniques. For this example, we'll do one epoch of training. Given the small size of the neural network relative to the size of the dataset, this should be sufficient to achieve comparable accuracy to what was observed in the DKL paper.

The optimization loop differs from the one seen in our more simple tutorials in that it involves looping over both a number of training iterations (epochs) *and* minibatches of the data. However, the basic process is the same: for each minibatch, we forward through the model, compute the loss (the `VariationalMarginalLogLikelihood` or ELBO), call backwards, and do a step of optimization.

In [5]:
from pyro import optim
from pyro import infer

optimizer = optim.Adam({"lr": 0.01})
num_epochs = 100

kernel = infer.RBFSteinKernel()
svi = infer.SVGD(model.model, kernel, optimizer, num_particles=16, max_plate_nesting=1)

for i in range(num_epochs):
    # Within each iteration, we will go over each minibatch of data
    loader = tqdm.tqdm_notebook(train_loader, desc=f"Train (Epoch {i + 1})")
    for x_batch, y_batch in loader:
        loss = svi.step(x_batch, y_batch)
        loader.set_postfix(
            loss=(loss['.inducing_values'] / train_n),
            ls=model.covar_module.base_kernel.lengthscale.item(),
            os=model.covar_module.outputscale.item()
        )


In [6]:
svi.get_named_particles()

{'.inducing_values': tensor([[[-0.9582, -1.8119,  2.5637,  ..., -0.3439, -2.8731,  0.0059]],
 
         [[ 0.5019, -0.6590,  0.3354,  ..., -0.2551, -3.2047,  0.0772]],
 
         [[-0.2549, -0.0914,  1.5766,  ..., -0.7665, -3.8475,  0.0601]],
 
         ...,
 
         [[-0.7416,  0.1338,  0.3840,  ..., -0.2235, -2.3070,  0.0142]],
 
         [[ 0.3613,  1.1421,  0.0617,  ..., -0.6045, -3.4545, -0.0721]],
 
         [[-0.2461,  0.5339, -0.3419,  ..., -0.5055, -3.2253,  0.0455]]],
        device='cuda:0', grad_fn=<ViewBackward>)}

## Making Predictions

The next cell gets the predictive covariance for the test set (and also technically gets the predictive mean, stored in `preds.mean()`). Because the test set is substantially smaller than the training set, we don't need to make predictions in mini batches here, although this can be done by passing in minibatches of `test_x` rather than the full tensor.

In [11]:
model.eval()
likelihood.eval()
all_means = []
all_vars = []
all_log_probs = []
with torch.no_grad(), pyro.condition(data=svi.get_named_particles()):
    for x_batch, y_batch in test_loader:
        preds = model.likelihood(model(x_batch))
        means = preds.mean.mean(dim=0).cpu()
        vars = preds.variance.mean(dim=0).cpu()
        log_probs = torch.distributions.Normal(
            means, vars.clamp_min(1e-5).sqrt()
        ).log_prob(y_batch.cpu())

        all_means.append(means)
        all_vars.append(vars)
        all_log_probs.append(log_probs)

means = torch.cat(all_means)
vars = torch.cat(all_vars)
log_probs = torch.cat(all_log_probs)

In [12]:
print('Test MAE: {}'.format(torch.mean(torch.abs(means - test_y.cpu()))))

Test MAE: 0.4570150077342987


In [13]:
print('Test NLL: {}'.format(-torch.mean(log_probs)))

Test NLL: 0.7716305255889893


In [14]:
print(vars.min(), vars.mean(), vars.max())

tensor(0.0063) tensor(0.3196) tensor(1.3654)
